## 신경망 모델(HGNN)

In [ ]:
import pandas as pd
import mysql.connector

db_connection = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='',
    db='hyperanalystic',
)

# 데이터 가져오기
query = "SELECT patent_name, processed_patent_name, country, application_year FROM train_predict_data;"
train_data = pd.read_sql(query, db_connection)

# 연결 종료
db_connection.close()

C:\Users\dnjsr\AppData\Local\Temp\ipykernel_95564\2789984928.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  train_data = pd.read_sql(query, db_connection)


                                         patent_name   
0                                    파장변환 및 클락 추출 방법  \
1                    사용자 및 디바이스 기반의 도메인 관리 방법 및 그 장치   
2  대기상태 전원 및 구동전압 제어기 및 그를 이용한 고주파 이동통신 송수신 회로 및 시스템   
3                           정보보호를 위한 알에프아이디 시스템 및 방법   
4                       무선 휴대 인터넷 시스템에서 채널 모드간 전환 방법   

                      processed_patent_name country  application_year  
0                                파장변환 추출 방법      미국              2006  
1                  사용자 디바이스 기반 도메인 관리 방법 장치      미국              2007  
2  대기상태 전원 구동전압 제어기 그 이용 고주파 이동통신 송수신 회 시스템      미국              2006  
3                        정보보호 알에프아이디 시스템 방법      미국              2006  
4                무선 휴대 인터넷 시스템 채널 모드간 전환 방법      미국              2006  


#### 학습

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# 단어 빈도 계산
all_words = [word for words in train_data['processed_patent_name'] for word in words]
word_freq = pd.Series(all_words).value_counts()

# 빈도 조건 설정
min_freq = 3  # 최소 빈도 조건
max_freq = 0.95 * len(train_data)  # 최대 빈도 조건

# 조건에 맞는 단어 필터링
filtered_words = word_freq[(word_freq >= min_freq) & (word_freq <= max_freq)].index.tolist()

# 필터링 결과 확인
print(f"필터링된 단어: {filtered_words}")
print(f"필터링된 단어 수: {len(filtered_words)}")

# 'processed_patent_name' 컬럼 업데이트
train_data['processed_patent_name'] = train_data['processed_patent_name'].apply(
    lambda words: [word for word in words if word in filtered_words]
)

# 필터링 후 빈 리스트를 대체
train_data['processed_patent_name'] = train_data['processed_patent_name'].apply(
    lambda words: words if len(words) > 0 else ['unknown']
)

# 결과 확인
empty_rows = train_data[train_data['processed_patent_name'].apply(len) == 0]
print(f"빈 리스트: {len(empty_rows)}")

# 인코딩
mlb = MultiLabelBinarizer()
encoded_data = mlb.fit_transform(train_data['processed_patent_name'])
print(f"Encoded shape: {encoded_data.shape}")

# 데이터 변환 결과 저장
train_data['encoded_patent_name'] = list(encoded_data)

필터링된 단어: ['방', '법', '이', '장', '치', '스', '시', '신', '기', '용', '호', '템', '화', '전', '터', '송', '상', '제', '리', '수', '트', '통', '선', '정', '부', '성', '자', '동', '지', '반', '무', '조', '다', '디', '영', '인', '적', '레', '보', '비', '크', '복', '오', '중', '서', '드', '파', '프', '구', '광', '체', '데', '그', '원', '대', '모', '사', '형', '공', '링', '소', '널', '차', '단', '로', '향', '가', '생', '간', '측', '채', '식', '계', '변', '워', '력', '분', '어', '속', '네', '고', '코', '출', '라', '관', '역', '버', '위', '주', '세', '안', '포', '경', '율', '유', '도', '환', '패', '테', '예', '저', '행', '랜', '처', '합', '티', '응', '작', '딩', '메', '미', '능', '플', '접', '나', '연', '검', '하', '록', '음', '블', '추', '센', '산', '한', '할', '재', '효', '물', '실', '색', '매', '발', '캐', '증', '함', '멀', '감', '탐', '에', '러', '킷', '층', '우', '입', '망', '면', '결', '당', '브', '운', '셀', '량', '바', '내', '회', '마', '츠', '션', '교', '일', '노', '길', '절', '설', '질', '직', '케', '객', '국', '벡', '텐', '양', '콘', '태', '타', '털', '개', '집', '폭', '초', '래', '액', '거', '더', '피', '류', '말', '압', '듈', '임', '점', '픽', '현', '석', '토', '배', '클', 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HypergraphConv, global_mean_pool
from torch_geometric.data import Data
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import torch.nn as nn

# 하이퍼그래프 모델 정의
class HGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(HGNN, self).__init__()
        self.layer1 = HypergraphConv(input_dim, hidden_dim)
        self.layer2 = HypergraphConv(hidden_dim, output_dim)

    def forward(self, x, edge_index, batch=None):
        if edge_index.size(1) == 0:
            print("빈 엣지 인덱스")
            return x
        x = F.relu(self.layer1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.layer2(x, edge_index)
        if batch is not None:
            x = global_mean_pool(x, batch)
        return x

# 데이터셋 분할
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

train_indices, test_indices = train_test_split(range(len(train_data)), test_size=test_ratio, random_state=42)
train_indices, val_indices = train_test_split(train_indices, test_size=val_ratio / (train_ratio + val_ratio), random_state=42)

train_data_split = train_data.iloc[train_indices]
val_data_split = train_data.iloc[val_indices]
test_data_split = train_data.iloc[test_indices]

# 하이퍼그래프 데이터 생성 함수
def create_hypergraph_data(data_split, mlb, num_nodes):
    edges = []
    for idx, row in data_split.iterrows():
        patent_name = row['processed_patent_name']
        for word in patent_name:
            try:
                word_index = mlb.classes_.tolist().index(word)
                edges.append((word_index, idx))
            except ValueError as e:
                print(f"Warning: Word '{word}' not in MultiLabelBinarizer classes.")
    if edges:
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    else:
        edge_index = torch.empty((2, 0), dtype=torch.long)
    x = torch.eye(num_nodes, dtype=torch.float)
    return Data(x=x, edge_index=edge_index)

num_nodes = len(mlb.classes_)
train_data_obj = create_hypergraph_data(train_data_split, mlb, num_nodes)
val_data_obj = create_hypergraph_data(val_data_split, mlb, num_nodes)
test_data_obj = create_hypergraph_data(test_data_split, mlb, num_nodes)

# 모델 초기화
model = HGNN(input_dim=num_nodes, hidden_dim=64, output_dim=2)
optimizer = Adam(model.parameters(), lr=0.04)

# 타겟 생성
train_target = torch.randint(0, 2, (train_data_obj.num_nodes, 2)).float()
val_target = torch.randint(0, 2, (val_data_obj.num_nodes, 2)).float()
test_target = torch.randint(0, 2, (test_data_obj.num_nodes, 2)).float()

# 학습 및 검증
def train_with_validation(train_data, val_data, model, optimizer, train_target, val_target, epochs=100):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(train_data.x, train_data.edge_index)
        train_loss = F.mse_loss(out, train_target)
        train_loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_out = model(val_data.x, val_data.edge_index)
            val_loss = F.mse_loss(val_out, val_target)

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Train Loss: {train_loss.item()}, Validation Loss: {val_loss.item()}")

train_with_validation(train_data_obj, val_data_obj, model, optimizer, train_target, val_target)

Epoch 0, Train Loss: 0.5147480368614197, Validation Loss: 0.2694413959980011
Epoch 10, Train Loss: 0.25180816650390625, Validation Loss: 0.267046183347702
Epoch 20, Train Loss: 0.260055810213089, Validation Loss: 0.26516908407211304
Epoch 30, Train Loss: 0.2495841532945633, Validation Loss: 0.25743842124938965
Epoch 40, Train Loss: 0.24594317376613617, Validation Loss: 0.2541368305683136
Epoch 50, Train Loss: 0.24522702395915985, Validation Loss: 0.25378715991973877
Epoch 60, Train Loss: 0.24496588110923767, Validation Loss: 0.25548964738845825
Epoch 70, Train Loss: 0.24202609062194824, Validation Loss: 0.2543905973434448
Epoch 80, Train Loss: 0.2395016998052597, Validation Loss: 0.25426480174064636
Epoch 90, Train Loss: 0.23490482568740845, Validation Loss: 0.2551967203617096


#### 예측

In [4]:
# 상위 특허 추출
model.eval()
with torch.no_grad():
    out = model(train_data_obj.x, train_data_obj.edge_index)

predicted_scores = out[:, 1]
top_10_indices = predicted_scores.argsort(descending=True)[:200]
top_10_patents = train_data.iloc[top_10_indices].copy()
top_10_scores = predicted_scores[top_10_indices]
top_10_patents['score'] = top_10_scores.numpy()

top_10_patents_unique = top_10_patents.groupby('patent_name').agg({
    'application_year': 'first',
    'country': 'first',
    'score': 'max',
}).reset_index()

top_10_patents_unique_filtered = top_10_patents_unique[top_10_patents_unique['application_year'] > 2016]
top_10_patents_unique_filtered = top_10_patents_unique_filtered.sort_values(by='score', ascending=False)

if top_10_patents_unique_filtered.empty:
    print("특허가 존재하지 않음")
else:
    for idx, row in top_10_patents_unique_filtered.iterrows():
        print(f"Rank {idx + 1}: Patent Name: {row['patent_name']}, Year: {row['application_year']}, "
              f"Country: {row['country']}, Potential Score: {row['score']:.4f}")

Rank 48: Patent Name: 다중 사용자 무선 통신 시스템에서 제어 및 훈련 심볼 전송 방법, Year: 2018, Country: 대한민국, Potential Score: 0.6128
Rank 113: Patent Name: 영상 부호화 및 복호화를 위한 장치 및 방법, Year: 2017, Country: 대한민국, Potential Score: 0.6011
Rank 153: Patent Name: 중앙 집중식 매체 접속 제어 프로토콜을 사용하는 광대역 고주파수 무선 시스템에서 우회 경로 설정 방법 및 그 장치, Year: 2019, Country: 미국, Potential Score: 0.5700
Rank 167: Patent Name: 통합 음성/음악 신호 부/복호화 방법 및 장치, Year: 2018, Country: 미국, Potential Score: 0.5565
Rank 163: Patent Name: 초고주파 송수신 장치, Year: 2019, Country: 미국, Potential Score: 0.5513
Rank 142: Patent Name: 적응적 스캐닝을 이용한 동영상 부호화/복호화 장치 및 그 방법, Year: 2018, Country: 미국, Potential Score: 0.5484
Rank 5: Patent Name: 60GHz 대역을 사용하는 LOS 통신시스템에서 중계 장치를 경유하여로 우회 경로를 통한 데이터 송수신 방법 및 그 장치, Year: 2018, Country: 미국, Potential Score: 0.5471
Rank 11: Patent Name: MBMS 서비스의 동적 스케줄링 지원 방법 및 절차, Year: 2018, Country: 미국, Potential Score: 0.5356


## 평가

In [5]:
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
import torch

# 테스트 데이터 평가 함수 (MAE, Accuracy)
def test(test_data, model, test_target):
    model.eval()
    with torch.no_grad():
        out = model(test_data.x, test_data.edge_index) 
        
        test_loss = F.mse_loss(out, test_target)
        print(f"Test Loss: {test_loss.item()}")

        # 예측 값과 실제 값 변환
        y_true = test_target.numpy()  # 실제 값
        y_pred = out.numpy()         # 예측 값

        # MAE 계산
        mae = mean_absolute_error(y_true, y_pred)

        # Accuracy
        # 예측 클래스와 실제 클래스 비교
        predicted_class = torch.argmax(out, dim=1)  # 가장 큰 값이 예측 클래스
        y_true_classes = y_true.argmax(axis=1)      # 실제 클래스

        accuracy = accuracy_score(y_true_classes, predicted_class.numpy())
        # 평가 결과 출력
        print(f"Mean Absolute Error (MAE): {mae:.4f}")
        print(f"Accuracy: {accuracy:.4f}")

# 테스트 평가 실행
test(test_data_obj, model, test_target)


Test Loss: 0.2583553194999695
Mean Absolute Error (MAE): 0.5005
Accuracy: 0.4951
